In [1]:
import pandas as pd
import os
import glob
from pathlib import Path

In [4]:
# Define the directory containing the CSV files
pdb_reports_dir = '/home/markus/MPI_local/data/PDB_reports/1'

# Get all CSV files in the directory, sorted to maintain order
csv_files = sorted(glob.glob(os.path.join(pdb_reports_dir, '*.csv')))

print(f"Found {len(csv_files)} CSV files in {pdb_reports_dir}")
for file in csv_files:
    print(f"  - {os.path.basename(file)}")

Found 5 CSV files in /home/markus/MPI_local/data/PDB_reports/1
  - rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_00001-02500.csv
  - rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_02501-05000.csv
  - rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_05001-07500.csv
  - rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_07501-10000.csv
  - rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_10001-12027.csv


In [13]:
# Read all CSV files and combine them into a single dataframe
combined_df = pd.DataFrame()

for file_path in csv_files:
    print(f"Reading {os.path.basename(file_path)}...")
    
    # Read each CSV file, skipping first row and using second row as headers
    df = pd.read_csv(file_path, skiprows=1, header=0)
    
    # Append to the combined dataframe
    combined_df = pd.concat([combined_df, df], ignore_index=True)

print(f"\nCombined dataframe shape: {combined_df.shape}")
print(f"Columns: {list(combined_df.columns)}")

# Display first few rows to understand the structure
print("\nFirst 5 rows:")
print(combined_df.head())

# Check for missing values in key columns
key_columns = ['Entry ID', 'Deposition Date', 'Release Date', 'PDB ID']
print("\nMissing values in key columns:")
for col in key_columns:
    if col in combined_df.columns:
        missing_count = combined_df[col].isna().sum()
        print(f"{col}: {missing_count} missing values")
    else:
        print(f"{col}: Column not found")

Reading rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_00001-02500.csv...
Reading rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_02501-05000.csv...
Reading rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_05001-07500.csv...
Reading rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_07501-10000.csv...
Reading rcsb_pdb_custom_report_600ea47da61adfe8edd637fec8227fc9_10001-12027.csv...

Combined dataframe shape: (43196, 15)
Columns: ['Entry ID', 'Deposition Date', 'Release Date', 'PDB ID', 'Sequence', 'Entity Polymer Type', 'Total Number of polymer Entity Instances (Chains) per Entity', 'Source Organism', 'Taxonomy ID', 'Gene Name', 'Entity ID', 'Entry Id (Polymer Entity Identifiers)', 'Accession Code(s)', 'Database Name', 'Unnamed: 14']

First 5 rows:
  Entry ID Deposition Date Release Date PDB ID  \
0     1NVP      2003-02-04   2003-10-21   1NVP   
1      NaN             NaN          NaN    NaN   
2      NaN             NaN          NaN    NaN   
3      NaN 

In [14]:
# Fill missing values in key columns using forward fill
# This fills each missing value with the last valid value that appeared before it
print("Filling missing values...")

# Make a copy of the dataframe for processing
processed_df = combined_df.copy()

# Define the columns that need forward filling
columns_to_fill = ['Entry ID', 'Deposition Date', 'Release Date', 'PDB ID']

# Apply forward fill to each column
for col in columns_to_fill:
    if col in processed_df.columns:
        # forward fill
        processed_df[col] = processed_df[col].ffill()

print("Missing values after filling:")
for col in columns_to_fill:
    if col in processed_df.columns:
        missing_count = processed_df[col].isna().sum()
        print(f"{col}: {missing_count} missing values")

# Display a sample of the processed data
print("\nSample of processed data:")
print(processed_df[columns_to_fill].head(10))

Filling missing values...
Missing values after filling:
Entry ID: 0 missing values
Deposition Date: 0 missing values
Release Date: 0 missing values
PDB ID: 0 missing values

Sample of processed data:
  Entry ID Deposition Date Release Date PDB ID
0     1NVP      2003-02-04   2003-10-21   1NVP
1     1NVP      2003-02-04   2003-10-21   1NVP
2     1NVP      2003-02-04   2003-10-21   1NVP
3     1NVP      2003-02-04   2003-10-21   1NVP
4     1NVP      2003-02-04   2003-10-21   1NVP
5     1NVP      2003-02-04   2003-10-21   1NVP
6     5GPY      2016-08-05   2016-11-02   5GPY
7     5GPY      2016-08-05   2016-11-02   5GPY
8     8CLL      2023-02-16   2023-06-21   8CLL
9     8CLL      2023-02-16   2023-06-21   8CLL


In [15]:
# Save the processed dataframe to a new CSV file
output_file = os.path.join(pdb_reports_dir, 'combined_pdb_reports_processed.csv')

print(f"Saving processed data to: {output_file}")
processed_df.to_csv(output_file, index=False)

print(f"Successfully saved {len(processed_df)} rows to {output_file}")
print(f"Final dataframe shape: {processed_df.shape}")

# Display summary statistics
print("\nSummary of the final processed dataset:")
print(f"- Total rows: {len(processed_df)}")
print(f"- Unique PDB IDs: {processed_df['PDB ID'].nunique() if 'PDB ID' in processed_df.columns else 'N/A'}")
print(f"- Date range: {processed_df['Deposition Date'].min() if 'Deposition Date' in processed_df.columns else 'N/A'} to {processed_df['Deposition Date'].max() if 'Deposition Date' in processed_df.columns else 'N/A'}")

Saving processed data to: /home/markus/MPI_local/data/PDB_reports/1/combined_pdb_reports_processed.csv
Successfully saved 43196 rows to /home/markus/MPI_local/data/PDB_reports/1/combined_pdb_reports_processed.csv
Final dataframe shape: (43196, 15)

Summary of the final processed dataset:
- Total rows: 43196
- Unique PDB IDs: 12027
- Date range: 1995-01-06 to 2025-06-29
Successfully saved 43196 rows to /home/markus/MPI_local/data/PDB_reports/1/combined_pdb_reports_processed.csv
Final dataframe shape: (43196, 15)

Summary of the final processed dataset:
- Total rows: 43196
- Unique PDB IDs: 12027
- Date range: 1995-01-06 to 2025-06-29
